In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
os.environ['DEBUG'] = '2'

import tinygrad
from tinygrad import Tensor



Reading Andriej's article and coding along.


In [2]:
t = tinygrad.Tensor(np.linspace(0,5,6), dtype='float', device='metal')
c = t * 3
d = c.realize()
d.numpy()

opened device NPY from pid:36318
opened device METAL from pid:36318
scheduled 2 kernels in 182.30 ms
*** METAL      1 copy       24,   METAL <- NPY                arg  2 mem  0.00 GB tm     61.67us/     0.06ms (     0.00 GFLOPS    0.0|0.0     GB/s) 
*** METAL      2 E_2_3                                        arg  2 mem  0.00 GB tm      6.88us/     0.07ms (     0.00 GFLOPS    0.0|0.0     GB/s) ['__mul__']


array([ 0.,  3.,  6.,  9., 12., 15.], dtype=float32)

In [3]:
def encoder( tokens):
  assert len(set(tokens)) == len(tokens), f'non-unique chars in tokens: {tokens}'
  map = {}
  rmap = {}
  for i,t in enumerate(tokens):
    map[t]=i
    rmap[i] = t
  def encode(data):
    result = np.zeros((len(tokens), len(data)))
    for i, d in enumerate(data):
      result[map[d], i] = 1
    return result
  def decode_one(data):
    assert isinstance(data, np.ndarray)
    for i,d in enumerate(data):
      if d != 0:
        return rmap[i] 
    
  return encode, decode_one

In [ ]:
# what are the dimensions of things?
'''
class RNN():
  def __init__(self, ):
    pass
  def step(self, x):
    # update the hidden state
    self.h = np.tanh(
      np.dot(self.W_hh, self.h) + np.dot(self.W_xh, x)
      )

    # compute the output vec
    y = np.dot(self.W_hy, self.h)
    return y
'''
# in here, we see that the hidden state length 

class RNN():
  def __init__(self, input_size, hidden_size, tokens ):
    self.batch_size = 1
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.encode, self.decode = encoder(tokens)
    self.h = Tensor(np.zeros((self.hidden_size)), dtype='float')
    self.W_xh = Tensor(np.random.uniform(-1, 1, (self.input_size, self.hidden_size) ), dtype='float', requires_grad=True)
    self.W_hh = Tensor(np.random.uniform(-1, 1, (self.hidden_size, self.hidden_size)) , dtype='float' , requires_grad=True)
    self.W_hy = Tensor(np.random.uniform(-1, 1, (self.hidden_size, self.input_size) ) , dtype='float', requires_grad=True) 

  def __repr__(self):
    return f'h:\n{self.h.numpy()}\n----------\nxh:\n{self.W_xh.numpy()}\n----------\nhh:\n{self.W_hh.numpy()}\n----------\nhy:\n{self.W_hy.numpy()}'
  
  def step(self, input):
    input = Tensor(np.asarray(input), dtype='float' )
    p1 = self.W_hh @ self.h.reshape(self.hidden_size, 1).flatten()
    p2 = (input.reshape(1, self.input_size) @ self.W_xh).flatten()
    self.h = (p1 + p2).tanh()
    y =  self.h @ self.W_hy
    return y
  
  def forward(self, input):
    enc_input = self.encode(input)
    result = []
    for col in enc_input.T:
      result.append(self.step(col).numpy()) 
    
    return ''.join([self.decode(r) for r in result])
  
  def train(self, input, epochs=1000):
      
    
  def backward(self, input):
    pass

  def predict(self, input):
    pass
  


In [5]:
Tensor(np.array([1,2,3])).reshape(3,1)

<Tensor <UOp METAL (3, 1) long ShapeTracker(views=(View(shape=(3, 1), strides=(1, 0), offset=0, mask=None, contiguous=True),))> on METAL with grad None>

In [6]:
r = RNN(4, 3, 'helo')
y = r.forward('hheelloo')
print(r)

import os


scheduled 8 kernels in 6.98 ms
*** METAL      3 copy       36,   METAL <- NPY                arg  2 mem  0.00 GB tm     68.83us/     0.14ms (     0.00 GFLOPS    0.0|0.0     GB/s) 
*** METAL      4 copy       12,   METAL <- NPY                arg  2 mem  0.00 GB tm     38.62us/     0.18ms (     0.00 GFLOPS    0.0|0.0     GB/s) ['__matmul__']
*** METAL      5 copy       16,   METAL <- NPY                arg  2 mem  0.00 GB tm     86.17us/     0.26ms (     0.00 GFLOPS    0.0|0.0     GB/s) 
*** METAL      6 copy       48,   METAL <- NPY                arg  2 mem  0.00 GB tm     29.37us/     0.29ms (     0.00 GFLOPS    0.0|0.0     GB/s) 
*** METAL      7 copy       48,   METAL <- NPY                arg  2 mem  0.00 GB tm     10.29us/     0.30ms (     0.00 GFLOPS    0.0|0.0     GB/s) 
*** METAL      8 r_3_4                                        arg  3 mem  0.00 GB tm      7.92us/     0.31ms (     0.00 GFLOPS    0.0|0.0     GB/s) ['__matmul__']
*** METAL      9 r_3_3                         

In [ ]:

x = Tensor([1,2,3] * 3, dtype='float', requires_grad=False).reshape((3,3))
w1 = Tensor([-1, 1, 1] * 3,dtype='float', requires_grad=True).reshape((3,3))
b = Tensor.uniform(3,3, low=0,high=1, dtype='float')
w2 = Tensor([2, 0.5, 1] * 3,dtype='float', requires_grad=True).reshape((3,3))

print(x.numpy(), w1.numpy(), b.numpy(), w2.numpy(), sep='\n\n')
y = (((x @ w1) + b) @ w2)
#y.backward()
y.gradient(w1, gradient=Tensor.ones_like(w1))[0].numpy()
#w1.grad.numpy()





scheduled 2 kernels in 4.91 ms
*** METAL     65 copy       36,   METAL <- PYTHON             arg  2 mem  0.00 GB tm     70.83us/     1.36ms (     0.00 GFLOPS    0.0|0.0     GB/s) 
*** METAL     66 E_3_3n2                                      arg  2 mem  0.00 GB tm      6.88us/     1.37ms (     0.00 GFLOPS    0.0|0.0     GB/s) ['reshape']
scheduled 2 kernels in 7.09 ms
*** METAL     67 copy       36,   METAL <- PYTHON             arg  2 mem  0.00 GB tm     25.25us/     1.39ms (     0.00 GFLOPS    0.0|0.0     GB/s) 
*** METAL     68 E_3_3n2                                      arg  2 mem  0.00 GB tm      7.21us/     1.40ms (     0.00 GFLOPS    0.0|0.0     GB/s) ['reshape']
scheduled 37 kernels in 89.37 ms
*** METAL     69 r_5_5                                        arg  1 mem  0.00 GB tm      6.13us/     1.41ms (     0.00 GFLOPS    0.0|0.0     GB/s) ['uniform']
*** METAL     70 copy        4,   METAL <- PYTHON             arg  2 mem  0.00 GB tm     30.00us/     1.44ms (     0.00 GFLOPS 

array([[10.5, 10.5, 10.5],
       [21. , 21. , 21. ],
       [31.5, 31.5, 31.5]], dtype=float32)